In [1]:
import torch
import pandas as pd
from PIL import Image
import torchvision.transforms as transforms
from torchvision import datasets, models
import numpy as np
import json
import requests
import matplotlib.pyplot as plt
import warnings
import os
warnings.filterwarnings('ignore')
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import torch.nn as nn
import torchvision
import matplotlib.pyplot as plt
from torch.utils.tensorboard import SummaryWriter
# %load_ext tensorboard
import datetime
import time

from torchvision.models.resnet import *
from torchvision.models.resnet import BasicBlock, Bottleneck

torch.cuda.empty_cache()

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)

cuda


In [2]:
'''
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)
os.chdir('drive/MyDrive/Colab Notebooks/AICORE/FAISS')
'''

"\nfrom google.colab import drive\ndrive.mount('/content/drive/', force_remount=True)\nos.chdir('drive/MyDrive/Colab Notebooks/AICORE/FAISS')\n"

In [3]:
## start the tensorboard
%load_ext tensorboard

In [4]:
%tensorboard --logdir {logs_base_dir}  --host localhost

In [5]:
#model = models.resnet50(pretrained=True)

In [6]:
writer = SummaryWriter(log_dir='logs')

In [7]:
os.getcwd()

'/home/ubuntu/facebook-marketplaces-recommendation-ranking-system/Practicals'

In [8]:
'''
import os

def remove_ds_store(root_dir):
    for root, _, files in os.walk(root_dir):
        for file in files:
            if file == ".DS_Store":
                file_path = os.path.join(root, file)
                os.remove(file_path)
                print(f"Removed: {file_path}")

# Replace 'path_to_your_directory' with the actual path to the directory you want to clean
directory_to_clean = 'aicore_images_split'
remove_ds_store(directory_to_clean)
'''

'\nimport os\n\ndef remove_ds_store(root_dir):\n    for root, _, files in os.walk(root_dir):\n        for file in files:\n            if file == ".DS_Store":\n                file_path = os.path.join(root, file)\n                os.remove(file_path)\n                print(f"Removed: {file_path}")\n\n# Replace \'path_to_your_directory\' with the actual path to the directory you want to clean\ndirectory_to_clean = \'aicore_images_split\'\nremove_ds_store(directory_to_clean)\n'

In [9]:
## pre process images for speed
'''
from PIL import Image

def resize_and_convert_to_rgb(image_path, target_size=(64, 64)):
    try:
        image = Image.open(image_path)
        image = image.resize(target_size)

        if image.mode != 'RGB':
            image = image.convert('RGB')

        image.save(image_path)
        #print(f"Processed: {image_path}")
    except Exception as e:
        print(f"Error processing {image_path}: {e}")

def process_subdirectories(root_dir):
    for root, _, files in os.walk(root_dir):
        for file in files:
            if file.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.gif')):
                image_path = os.path.join(root, file)

                resize_and_convert_to_rgb(image_path)

root_directory = "aicore_images_split"
process_subdirectories(root_directory)

'''

'\nfrom PIL import Image\n\ndef resize_and_convert_to_rgb(image_path, target_size=(64, 64)):\n    try:\n        image = Image.open(image_path)\n        image = image.resize(target_size)\n\n        if image.mode != \'RGB\':\n            image = image.convert(\'RGB\')\n\n        image.save(image_path)\n        #print(f"Processed: {image_path}")\n    except Exception as e:\n        print(f"Error processing {image_path}: {e}")\n\ndef process_subdirectories(root_dir):\n    for root, _, files in os.walk(root_dir):\n        for file in files:\n            if file.lower().endswith((\'.jpg\', \'.jpeg\', \'.png\', \'.bmp\', \'.gif\')):\n                image_path = os.path.join(root, file)\n\n                resize_and_convert_to_rgb(image_path)\n\nroot_directory = "aicore_images_split"\nprocess_subdirectories(root_directory)\n\n'

In [10]:
## Creating a train dataset class
class ItemsTrainDataSet(Dataset):
    def __init__(self):
        super().__init__()
        self.examples = self._load_examples()
        #self.resize = transforms.Resize(64)
        self.randomhflip = transforms.RandomHorizontalFlip()
        #self.randomrot = transforms.RandomRotation(degrees=30)
        self.pil_to_tensor = transforms.ToTensor()
        self.normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

        #self.resize = transforms.Resize((225,225))
        #self.rgbify = transforms.Lambda(lambda x: x.repeat(3, 1, 1) if x.size(0)==1 else x)

    def _load_examples(self):
        class_names = os.listdir('aicore_images_split/train')
        class_encoder = {class_name: idx for idx, class_name in enumerate(class_names)}
        class_decoder = {idx: class_name for idx, class_name in enumerate(class_names)}

        examples_list = []
        for cl_name in class_names:
            example_fp = os.listdir(os.path.join('aicore_images_split/train',cl_name))
            example_fp = [os.path.join('aicore_images_split/train', cl_name, img_name ) for img_name in example_fp]
            example = [(img_name, class_encoder[cl_name]) for img_name in example_fp]
            examples_list.extend(example)

        return examples_list

    def __getitem__(self, idx):
        img_fp, img_class = self.examples[idx]
        img = Image.open(img_fp)
        img = self.randomhflip(img)
        #img = self.randomrot(img)

        #img = self.resize(img)
        features = self.pil_to_tensor(img)
        features = self.normalize(features)

        #features = self.rgbify(features)

        return features, img_class

    def __len__(self):
        return len(self.examples)

In [11]:
## Creates a validation dataset class
class ItemsValDataSet(Dataset):
    def __init__(self):
        super().__init__()
        self.examples = self._load_examples()
        #self.resize = transforms.Resize(64)
        self.pil_to_tensor = transforms.ToTensor()
        self.normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        #self.resize = transforms.Resize((225,225))
        #self.rgbify = transforms.Lambda(lambda x: x.repeat(3, 1, 1) if x.size(0)!=1 else x)

    def _load_examples(self):
        class_names = os.listdir('aicore_images_split/val')
        class_encoder = {class_name: idx for idx, class_name in enumerate(class_names)}
        class_decoder = {idx: class_name for idx, class_name in enumerate(class_names)}
        examples_list = []

        for cl_name in class_names:
            example_fp = os.listdir(os.path.join('aicore_images_split/val',cl_name))
            example_fp = [os.path.join('aicore_images_split/val', cl_name, img_name ) for img_name in example_fp]
            example = [(img_name, class_encoder[cl_name]) for img_name in example_fp]
            examples_list.extend(example)

        return examples_list

    def __getitem__(self, idx):
        img_fp, img_class = self.examples[idx]
        img = Image.open(img_fp)

        #img = self.resize(img)
        features = self.pil_to_tensor(img)
        features = self.normalize(features)

        return features, img_class

    def __len__(self):
        return len(self.examples)

In [12]:
traindataset = ItemsTrainDataSet()
len(traindataset)

8816

In [13]:
valdataset = ItemsValDataSet()
len(valdataset)

3788

In [14]:
## Created a classifier based on the RESNET50 pretrained model

class ItemClassifier(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.resnet50 = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_resnet50', pretrained=True)
        self.resnet50.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.resnet50.avgpool = nn.AdaptiveAvgPool2d(1)
        #self.resnet50 = model
        self.resnet50.fc = torch.nn.Linear(2048,13)

    def forward(self, X):
        return F.softmax(self.resnet50(X))

In [15]:
classifier = ItemClassifier()

Using cache found in /home/ubuntu/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub


In [16]:
'''
def train(model,traindataloader, valdataloader, epochs):
    #optimiser = torch.optim.SGD(model.parameters(), lr=0.001)
    #optimiser = torch.optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0)
    optimiser = torch.optim.Adam(model.parameters(), lr=0.01, betas=(0.9, 0.999), eps=1e-08, weight_decay=0)
    #optimiser = torch.optim.AdamW(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0.01)
    model_path = str(os.path.join('model_evaluation', time.strftime("%Y%m%d-%H%M%S")))
    os.makedirs(model_path)
    os.makedirs(os.path.join(model_path, 'weights'))

    global_step = 0

    for epoch in range(epochs):
        training_loss = 0.0
        validation_loss = 0.0
        model.to(device)
        model.train()
        tr_num_correct = 0
        tr_num_examples = 0
        epoch_combo = 'epoch' + str(epoch)
        os.makedirs(os.path.join(model_path, 'weights', epoch_combo))
        for inputs, labels in traindataloader:
            #labels = labels.unsqueeze(1)
            #labels = labels.float()
            inputs = inputs.to(device)
            labels = labels.to(device)
            predictions = model(inputs)
            #print(predictions.shape)
            #print(labels.shape)
            loss = torch.nn.CrossEntropyLoss()
            loss = loss(predictions, labels)
            loss.backward()
            optimiser.step()
            model_save_dir = str(os.path.join(model_path, 'weights', epoch_combo, 'weights.pt'))
            full_path = str(os.getcwd())
            #print(model_save_dir)
            #torch.save({'epoch': epoch,
            #    'model_state_dict': model.state_dict(),
            #    'optimizer_state_dict': optimiser.state_dict()},
                  #str(os.path.join(full_path, model_save_dir)))
            #           model_save_dir)
            torch.save({'epoch': epoch,
                  'model_state_dict': model.state_dict(),
                  'optimizer_state_dict': optimiser.state_dict()},
                  str(os.path.join(full_path, model_save_dir)))

            optimiser.zero_grad()
            #batch_idx += 1
            training_loss += loss.item() * inputs.size(0)
            correct = torch.eq(torch.max(F.softmax(predictions, dim=1), dim=1)[1], labels)
            tr_num_correct += torch.sum(correct).item()
            tr_num_examples += correct.shape[0]
        training_loss /= len(traindataloader.dataset)
        training_accuracy = tr_num_correct / tr_num_examples
        ## add training performance to tensorboard
        writer.add_scalar('Training Loss', training_loss, global_step)
        writer.add_scalar('Training Accuracy', training_accuracy, global_step)

        model.eval()
        val_num_correct = 0
        val_num_examples = 0
        for inputs, labels in valdataloader:
            #labels = labels.unsqueeze(1)
            #labels = labels.float()
            inputs = inputs.to(device)
            labels = labels.to(device)
            predictions = model(inputs)
            loss = torch.nn.CrossEntropyLoss()
            loss = loss(predictions, labels)
            validation_loss += loss.item() * inputs.size(0)
            correct = torch.eq(torch.max(F.softmax(predictions, dim =1), dim=1)[1], labels)
            val_num_correct += torch.sum(correct).item()
            val_num_examples += correct.shape[0]
        validation_loss /= len(valdataloader.dataset)
        validation_accuracy = val_num_correct / val_num_examples
        ## add validation performance to tensorboard
        writer.add_scalar('Validation Loss', validation_loss, global_step)
        writer.add_scalar('Validation Accuracy', validation_accuracy, global_step)
        perf_dict = {}
        perf_dict[epoch] = {'training_loss': training_loss,
                            'val_loss': validation_loss,
                            'training_accuracy': tr_num_correct / tr_num_examples,
                            'val_accuracy': val_num_correct / val_num_examples}


        print('Epoch: {}, Training Loss: {:.2f}, Validation Loss: {:.2f}, train_accuracy = {:.2f},val_accuracy = {:.2f} '.format(epoch, training_loss, validation_loss, tr_num_correct / tr_num_examples,
                                                                                                                             val_num_correct / val_num_examples))
        global_step += 1
'''

'\ndef train(model,traindataloader, valdataloader, epochs):\n    #optimiser = torch.optim.SGD(model.parameters(), lr=0.001)\n    #optimiser = torch.optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0)\n    optimiser = torch.optim.Adam(model.parameters(), lr=0.01, betas=(0.9, 0.999), eps=1e-08, weight_decay=0)\n    #optimiser = torch.optim.AdamW(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0.01)\n    model_path = str(os.path.join(\'model_evaluation\', time.strftime("%Y%m%d-%H%M%S")))\n    os.makedirs(model_path)\n    os.makedirs(os.path.join(model_path, \'weights\'))\n\n    global_step = 0\n\n    for epoch in range(epochs):\n        training_loss = 0.0\n        validation_loss = 0.0\n        model.to(device)\n        model.train()\n        tr_num_correct = 0\n        tr_num_examples = 0\n        epoch_combo = \'epoch\' + str(epoch)\n        os.makedirs(os.path.join(model_path, \'weights\', epoch_combo))\n        for inputs,

In [17]:
for param in classifier.resnet50.parameters():
    param.requires_grad = False

## define the layers to unfreeze and then retrain
layers_to_unfreeze = ['layers.3']

for name, param in classifier.resnet50.named_parameters():
    for layer_name in layers_to_unfreeze:
        if layer_name in name:
            param.requires_grad = True
            break


In [18]:
'''
def train(model,traindataloader, valdataloader, epochs):
    #optimiser = torch.optim.SGD(model.parameters(), lr=0.001)
    #optimiser = torch.optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0)
    #optimiser = torch.optim.AdamW(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0.01)
    optimiser = torch.optim.SGD(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001, momentum=0.9, weight_decay=1e-4)

    model_path = str(os.path.join('model_evaluation', time.strftime("%Y%m%d-%H%M%S")))
    os.makedirs(model_path)
    os.makedirs(os.path.join(model_path, 'weights'))

    global_step = 0

    for epoch in range(epochs):
        training_loss = 0.0
        validation_loss = 0.0
        model.to(device)
        model.train()
        tr_num_correct = 0
        tr_num_examples = 0
        epoch_combo = 'epoch' + str(epoch)
        os.makedirs(os.path.join(model_path, 'weights', epoch_combo))
        for inputs, labels in traindataloader:
            #labels = labels.unsqueeze(1)
            #labels = labels.float()
            inputs = inputs.to(device)
            labels = labels.to(device)
            predictions = model(inputs)
            #print(predictions.shape)
            #print(labels.shape)
            loss = torch.nn.CrossEntropyLoss()
            loss = loss(predictions, labels)
            loss.backward()
            optimiser.step()
            model_save_dir = str(os.path.join(model_path, 'weights', epoch_combo, 'weights.pt'))
            full_path = str(os.getcwd())
            #print(model_save_dir)
            #torch.save({'epoch': epoch,
            #    'model_state_dict': model.state_dict(),
            #    'optimizer_state_dict': optimiser.state_dict()},
                  #str(os.path.join(full_path, model_save_dir)))
            #           model_save_dir)
            torch.save({'epoch': epoch,
                  'model_state_dict': model.state_dict(),
                  'optimizer_state_dict': optimiser.state_dict()},
                  str(os.path.join(full_path, model_save_dir)))

            optimiser.zero_grad()
            #batch_idx += 1
            training_loss += loss.item() * inputs.size(0)
            correct = torch.eq(torch.max(F.softmax(predictions, dim=1), dim=1)[1], labels)
            tr_num_correct += torch.sum(correct).item()
            tr_num_examples += correct.shape[0]
        training_loss /= len(traindataloader.dataset)
        training_accuracy = tr_num_correct / tr_num_examples
        ## add training performance to tensorboard
        writer.add_scalar('Training Loss', training_loss, global_step)
        writer.add_scalar('Training Accuracy', training_accuracy, global_step)

        model.eval()
        val_num_correct = 0
        val_num_examples = 0
        for inputs, labels in valdataloader:
            #labels = labels.unsqueeze(1)
            #labels = labels.float()
            inputs = inputs.to(device)
            labels = labels.to(device)
            predictions = model(inputs)
            loss = torch.nn.CrossEntropyLoss()
            loss = loss(predictions, labels)
            validation_loss += loss.item() * inputs.size(0)
            correct = torch.eq(torch.max(F.softmax(predictions, dim =1), dim=1)[1], labels)
            val_num_correct += torch.sum(correct).item()
            val_num_examples += correct.shape[0]
        validation_loss /= len(valdataloader.dataset)
        validation_accuracy = val_num_correct / val_num_examples
        ## add validation performance to tensorboard
        writer.add_scalar('Validation Loss', validation_loss, global_step)
        writer.add_scalar('Validation Accuracy', validation_accuracy, global_step)
        perf_dict = {}
        perf_dict[epoch] = {'training_loss': training_loss,
                            'val_loss': validation_loss,
                            'training_accuracy': tr_num_correct / tr_num_examples,
                            'val_accuracy': val_num_correct / val_num_examples}


        print('Epoch: {}, Training Loss: {:.2f}, Validation Loss: {:.2f}, train_accuracy = {:.2f},val_accuracy = {:.2f} '.format(epoch, training_loss, validation_loss, tr_num_correct / tr_num_examples,
                                                                                                                             val_num_correct / val_num_examples))
        global_step += 1
'''

'\ndef train(model,traindataloader, valdataloader, epochs):\n    #optimiser = torch.optim.SGD(model.parameters(), lr=0.001)\n    #optimiser = torch.optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0)\n    #optimiser = torch.optim.AdamW(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0.01)\n    optimiser = torch.optim.SGD(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001, momentum=0.9, weight_decay=1e-4)\n\n    model_path = str(os.path.join(\'model_evaluation\', time.strftime("%Y%m%d-%H%M%S")))\n    os.makedirs(model_path)\n    os.makedirs(os.path.join(model_path, \'weights\'))\n\n    global_step = 0\n\n    for epoch in range(epochs):\n        training_loss = 0.0\n        validation_loss = 0.0\n        model.to(device)\n        model.train()\n        tr_num_correct = 0\n        tr_num_examples = 0\n        epoch_combo = \'epoch\' + str(epoch)\n        os.makedirs(os.path.join(model_path, \'weights\', epoch_combo

In [19]:
## variation in training loop

def train(model, traindataloader, valdataloader, epochs):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    writer = SummaryWriter()
    #optimizer = torch.optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0)
    #optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=1e-4)
    optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=1e-4)
    
    model.to(device)
    criterion = torch.nn.CrossEntropyLoss()
    model_path = os.path.join('model_evaluation', time.strftime("%Y%m%d-%H%M%S"))
    os.makedirs(model_path)
    os.makedirs(os.path.join(model_path, 'weights'))
    global_step = 0

    for epoch in range(epochs):
        training_loss = 0.0
        validation_loss = 0.0
        tr_num_correct = 0
        tr_num_examples = 0
        epoch_combo = 'epoch' + str(epoch)
        os.makedirs(os.path.join(model_path, 'weights', epoch_combo))

        model.train()
        for inputs, labels in traindataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            predictions = model(inputs)
            loss = criterion(predictions, labels)
            loss.backward()
            optimizer.step()
            
            training_loss += loss.item() * inputs.size(0)
            correct = torch.eq(torch.max(F.softmax(predictions, dim=1), dim=1)[1], labels)
            tr_num_correct += torch.sum(correct).item()
            tr_num_examples += correct.shape[0]

        training_loss /= len(traindataloader.dataset)
        training_accuracy = tr_num_correct / tr_num_examples
        writer.add_scalar('Training Loss', training_loss, global_step)
        writer.add_scalar('Training Accuracy', training_accuracy, global_step)

        model.eval()
        val_num_correct = 0
        val_num_examples = 0
        with torch.no_grad():
            for inputs, labels in valdataloader:
                inputs, labels = inputs.to(device), labels.to(device)
                predictions = model(inputs)
                loss = criterion(predictions, labels)
                validation_loss += loss.item() * inputs.size(0)
                correct = torch.eq(torch.max(F.softmax(predictions, dim=1), dim=1)[1], labels)
                val_num_correct += torch.sum(correct).item()
                val_num_examples += correct.shape[0]

        validation_loss /= len(valdataloader.dataset)
        validation_accuracy = val_num_correct / val_num_examples
        writer.add_scalar('Validation Loss', validation_loss, global_step)
        writer.add_scalar('Validation Accuracy', validation_accuracy, global_step)

        print('Epoch: {}, Training Loss: {:.2f}, Validation Loss: {:.2f}, train_accuracy = {:.2f}, val_accuracy = {:.2f}'.format(epoch, training_loss, validation_loss, training_accuracy, validation_accuracy))

        # Save the model checkpoint at the end of each epoch
        model_save_dir = os.path.join(model_path, 'weights', epoch_combo, 'weights.pt')
        torch.save({'epoch': epoch,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict()},
                    model_save_dir)

        global_step += 1

    writer.close()


In [20]:
'''
## unfreeze last two layers
for param in classifier.resnet50.layer3:
  param.requires_grad=True
'''



'\n## unfreeze last two layers\nfor param in classifier.resnet50.layer3:\n  param.requires_grad=True\n'

In [21]:
'''
## define the layers to unfreeze and then retrain

layers_to_unfreeze = ['layers.2', 'layers.3']

for name, param in classifier.resnet50.named_parameters():
    for layer_name in layers_to_unfreeze:
        if layer_name in name:
            param.requires_grad = True
            break
'''

"\n## define the layers to unfreeze and then retrain\n\nlayers_to_unfreeze = ['layers.2', 'layers.3']\n\nfor name, param in classifier.resnet50.named_parameters():\n    for layer_name in layers_to_unfreeze:\n        if layer_name in name:\n            param.requires_grad = True\n            break\n"

In [ ]:
#train_dataset = ItemsTrainDataSet()
#val_dataset = ItemsValDataSet()
train_loader = DataLoader(dataset = traindataset, batch_size=32)
val_loader = DataLoader(dataset = valdataset, batch_size=32)
train(classifier, traindataloader= train_loader, valdataloader= val_loader, epochs=500)


Epoch: 0, Training Loss: 2.57, Validation Loss: 2.56, train_accuracy = 0.08, val_accuracy = 0.09
Epoch: 1, Training Loss: 2.56, Validation Loss: 2.56, train_accuracy = 0.09, val_accuracy = 0.08
Epoch: 2, Training Loss: 2.56, Validation Loss: 2.56, train_accuracy = 0.09, val_accuracy = 0.09
Epoch: 3, Training Loss: 2.56, Validation Loss: 2.56, train_accuracy = 0.09, val_accuracy = 0.09
Epoch: 4, Training Loss: 2.56, Validation Loss: 2.56, train_accuracy = 0.09, val_accuracy = 0.09
Epoch: 5, Training Loss: 2.56, Validation Loss: 2.56, train_accuracy = 0.10, val_accuracy = 0.10
Epoch: 6, Training Loss: 2.56, Validation Loss: 2.56, train_accuracy = 0.10, val_accuracy = 0.09
Epoch: 7, Training Loss: 2.56, Validation Loss: 2.56, train_accuracy = 0.10, val_accuracy = 0.09
Epoch: 8, Training Loss: 2.56, Validation Loss: 2.56, train_accuracy = 0.11, val_accuracy = 0.09
Epoch: 9, Training Loss: 2.56, Validation Loss: 2.56, train_accuracy = 0.11, val_accuracy = 0.09
Epoch: 10, Training Loss: 2.56

In [ ]:
#atch size 32 - slow but good